# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.\
Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что в данных имеются пропуски, а также в количестве дней трудового стажа есть отрицательные значения.\
Переходим к предобработке, где заполним пропуски, исправим аномальные значения, и проверим на наличие дубликатов.

## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе.\
Второй столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных есть аномалии в виде отрицательного значения количества дней трудового стажа в столбце `days_employed`.\
Обработаем значения в этом столбце: поменяем знак всем отрицательным числам на положительный.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения.\
Исправить такие значения сложно, поэтому оставляем их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения: `-1` и `20`.\
Удалим строки, в которых встречаются такие артефакты из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены.**

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный для более удобного анализа.**

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`.\
В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.\
Приведём их к строчным буквам.**

In [13]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных.\
Если такие строки присутствуют - удалим их.**

In [14]:
data.duplicated().sum()

71

In [15]:
data = data.drop_duplicates()

### Категоризация данных

**Создадим в датафрейме `data` столбец `total_income_category` с категориями по уровню дохода:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income=0):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Видим, что присутствуют следующие цели в разных формулировках:**
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Создадим функцию, с помощью которой сформируем новый столбец `purpose_category`, где укажем категорию цель взятия кредита.**

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

###  Исследуем данные и ответим на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
temp = data.copy()
# создадим функцию для вывода сводных
def que(category):
    data_temp = temp.pivot_table(index=category, values='debt', aggfunc=['count', 'sum', 'mean'])
    data_temp.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
    display(data_temp.style.format("{:.3f}").background_gradient(cmap='Blues', axis=0))

In [22]:
que('children')

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14091.000,1063.000,0.075
1,4808.000,444.000,0.092
2,2052.000,194.000,0.095
3,330.000,27.000,0.082
4,41.000,4.000,0.098
5,9.000,0.000,0.000


**Вывод:**

Многодетных семей слишком маленькая выборка, чтобы делать окончательный вердикт.\
По текущим данным можно подвести такой итог:
- Заемщики с количеством детей от 1 до 4 - имеют высокий процент долга по кредиту.
- У заемщиков с 5 детьми не было долгов по кредитам - все 9 дебиторов вернули кредит в срок.
- Заемщики без детей - имеют средний показатель долга по кредитам.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
que('family_status')

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796.000,273.000,0.098
в разводе,1189.000,84.000,0.071
вдовец / вдова,951.000,63.000,0.066
гражданский брак,4134.000,385.000,0.093
женат / замужем,12261.000,927.000,0.076


**Вывод:**
- Дебиторы официально одинокие, но ранее вступавшие в брак (с семейным статусом "в разводе" и "вдовцы/вдовы") - имеют самый низкий процент по задолженности кредита.
- Заемщики, которые ещё не вступали в брак или находятся в гражданском браке - имеют самый высокий процент долга по кредиту.
- Люди, которые замужем/женаты - имеют средний показатель долга по сравнению с остальными категориями.

Можно сделать заключение, что оформленные семейные отношения или их прошлый опыт налагают определенную ответственность и приучают к финансовой дисциплине.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Разобьём выборку на 5 равных частей, т.к. категории доходов из предобработки имеют несбалансированные размеры.

In [24]:
temp['total_income_grouped'] = pd.qcut(data['total_income'], q=5)
data_temp = temp.pivot_table(index='total_income_grouped', values='debt', aggfunc=['count', 'sum', 'mean'])
data_temp.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
# Оформим таблицу цветным градиентом, но можно ее вывести и просто display(data_temp)
display(data_temp.style.format("{:.3f}").background_gradient(cmap='Blues', axis=0))

,Всего кредитополучателей,Всего должников,Доля должников
total_income_grouped,,,
"(20666.999, 98514.0]",4267.000,344.000,0.081
"(98514.0, 132113.0]",4266.000,358.000,0.084
"(132113.0, 161380.0]",4266.000,373.000,0.087
"(161380.0, 214604.0]",4266.000,358.000,0.084
"(214604.0, 2265604.0]",4266.000,299.000,0.070


**Вывод**

- Самыми надёжными дебиторами оказались высокодоходные клиенты.
- Остальные категории кредитополучателей имеют схожую долю должников.

#### Как разные цели кредита влияют на его возврат в срок?

In [25]:
que('purpose_category')

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с автомобилем,4279.000,400.000,0.093
операции с недвижимостью,10751.000,780.000,0.073
получение образования,3988.000,369.000,0.093
проведение свадьбы,2313.000,183.000,0.079


**Вывод:**
- Дебиторы, бравшие кредит на операции с автомобилем и получение образования - имеют выше шанс вернуть кредит не вовремя.
- Заемщики, которые получали кредит на проведение свадьбы и операции с недвижимостью - шанс задолженности по кредиту меньше.

### Общий вывод

В данном проекте подготовил данные для анализа:
- обработал количественные пропуски
- привёл строковые значения к одному регистру
- убрал дубли
- объединил значения по категориям/диапазонам значений.

Проанализировал как влияют разные показатели на своевременный возврат кредита:
- По количеству детей:
    * все кредиты платят вовремя заемщики с 5 детьми.
    * больше всего задолженности - заемщики с 1-4 ребёнком.

- По семейному положению:
    * самый низкий процент по задолженности - с семейным статусом "в разводе" и "вдовцы/вдовы".
    * самый высокий процент - которые ещё не вступали в брак или находятся в гражданском браке.

- По уровню дохода:
    * Самыми надёжными дебиторами оказались высокодоходные клиенты. 
    * Остальные категории кредитополучателей имеют схожую долю должников.

- По цели кредита:
    * Дебиторы, бравшие кредит на операции с автомобилем и получение образования - чаще не вовремя возвращали кредит.
    * Заемщики, которые получали кредит на проведение свадьбы и операции с недвижимостью - меньше шанс задолженности по кредиту.